In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


## Load and Normalize the Data
---
We will load the data from the included csv file "csgo_round_snapshots.csv".


In [ ]:
import pandas as pd

# percentage of training set to use as validation
valid_size = 0.2


#load the data into a dataframe using pandas
df = pd.read_csv('csgo_round_snapshots.csv')

# obtain training indices that will be used for validation
num_train = len(df)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

#now we must seperate the features from the labels
features = df.drop('round_winner', axis=1)
labels   = df['round_winner']




#perform one hot encoding on the map column
features = pd.get_dummies(features,columns=['map'], drop_first=True)

#Normalize all the numerical data types 'minmax' 
normalization_func = 'minmax'

for column in features.columns:
    #skips if column is a boolean
    if pd.api.types.is_bool_dtype(features[column]):
        continue

    #skips if column does not change in value
    if features[column].max() - features[column].min() == 0:
        continue
    
    if normalization_func == 'minmax':
        features[column] = (features[column] - features[column].min()) / (features[column].max() - features[column].min())
    elif normalization_func == 'z_score':
        features[column] = (features[column] - features[column].mean()) / features[column].std()
    else:
        continue

labels = labels.replace({'CT': 0, 'T': 1})

#had trouble with boolean type conversion now we implicitly change bools to floats
for column in features.select_dtypes(include='bool').columns:
    features[column] = features[column].astype(float)

features_np = features.values
labels_np = labels.values


features_tensor = torch.tensor(features_np, dtype=torch.float64)
labels_tensor = torch.tensor(labels_np, dtype=torch.float64).unsqueeze(1)

dataset = TensorDataset(features_tensor, labels_tensor)

data_loader = DataLoader(dataset, shuffle=True)


## Define the Architecture

---

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        #defining the layers in the NN
        self.output = nn.Linear(102,1)

    
    def forward(self, x):

        x = F.sigmoid(self.output(x))
        return x


model = Net()
print(model)

#move tensors to GPU if CUDA is available

if train_on_gpu:
    model.cuda()
model = model.double()
        

Net(
  (output): Linear(in_features=102, out_features=1, bias=True)
)


## Specify Loss Function and Optimizer


In [4]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.BCELoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

## Training the Network


In [ ]:
n_epochs = 100

valid_loss_min = np.Inf




for epoch in range(1, n_epochs + 1):
    train_loss = 0.0
    valid_loss = 0.0

    model.train()
    
    model.train()
    for batch_idx, (data, target) in enumerate(data_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)

ValueError: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.